In [77]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import itertools
import geopandas as gpd

In [45]:
simplify_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'
gtfs_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/'

In [10]:
simplify_dir = 'D:/RST/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'

gtfs_dir = 'D:/RST/notebook/GTFS/TTC_2016-10-03/'

In [91]:
ward = gpd.read_file('/Volumes/Data2/RST/notebook/GIS/wards_no-island.geojson')

ward = ward[['AREA_SHORT_CODE', 'geometry']]
ward = ward.to_crs('epsg:26917')
ward['area'] = ward['geometry'].area/1000000

ward = ward.rename(columns = {'AREA_SHORT_CODE':'ward'})

In [46]:
ward_int = pd.read_csv('/Volumes/Data2/RST/notebook/GIS/int_wards.csv')

#ward_int = pd.read_csv('D:/RST/notebook/GIS/int_wards.csv')

ward_int['INT_ID_STR'] = ward_int['INT_ID'].astype(str)

In [60]:
period = 'AM'
period_hr = 7

In [48]:
# G = nx.read_gexf('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.gexf')
# graph_data = pd.read_csv('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.csv')


G = nx.read_gexf('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16.gexf')
graph_data = pd.read_csv('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16.csv')



stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

In [53]:
route_dir = stop_times_od[stop_times_od['hr_o'] == period_hr]
route_dir = route_dir[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()

#route_dir = route_dir.merge(graph_data)

In [54]:
route_dir = route_dir.append(route_dir.rename(columns = {'INT_ID_o':'INT_ID_d', 'INT_ID_d':'INT_ID_o'})[['INT_ID_o', 
                                                                            'INT_ID_d', 'route_short_name']])

route_dir = route_dir.drop_duplicates()


route_dir = route_dir.groupby(['INT_ID_o', 'INT_ID_d'])['route_short_name'].apply(list).reset_index()
route_dir = route_dir.merge(graph_data)

In [55]:
def func(row):

    single_cycle = row.copy()
    single_cycle.append(single_cycle[0])
    single_cycle.append(None)


    cycle_pairs = list(zip(single_cycle [::1], single_cycle [1::1]))[:-1]
    cycle_pairs = [list(map(int, sublist)) for sublist in cycle_pairs]
    cycle_df = route_dir.merge(pd.DataFrame.from_records(cycle_pairs, columns = ['INT_ID_o', 'INT_ID_d']))

    first_route = cycle_df.iloc[0,2]

    #check for cycles where it can be traversed by 1 route, ie eliminate out and back trips
    for route in first_route:
        if len(cycle_df[cycle_df['route_short_name'].apply(lambda x: route in x)]) == len(cycle_df):
            return ward,0,0,1
            break
        else:
            return ward, 1, cycle_df['scaled_freq'].min(), 0
        

In [73]:
result_list = []
ward_n = []

for ward in range(1,26,1):

    G_ward = G.copy()


    G_ward.remove_nodes_from(list(ward_int[ward_int['AREA_SHORT_CODE'] != ward]['INT_ID_STR']))

    N = G_ward.size()

    #find cycles
    cycle_list = list(nx.cycle_basis(G_ward))
    print('Ward ' + str(ward) + ', Size ' + str(N) + ' Found '+ str(len(cycle_list)) + ' cycles')

    temp_list = list(map(func, cycle_list))
    

    result_list.append(temp_list)
    
    if len(cycle_list) == 0:
        result_list.append([(ward, 0, 0, 0)])
    
    ward_n.append([ward, N])

    

Ward 1, Size 152 Found 14 cycles
Ward 2, Size 151 Found 6 cycles
Ward 3, Size 154 Found 6 cycles
Ward 4, Size 89 Found 8 cycles
Ward 5, Size 159 Found 11 cycles
Ward 6, Size 103 Found 2 cycles
Ward 7, Size 95 Found 4 cycles
Ward 8, Size 120 Found 5 cycles
Ward 9, Size 87 Found 6 cycles
Ward 10, Size 50 Found 3 cycles
Ward 11, Size 73 Found 7 cycles
Ward 12, Size 54 Found 1 cycles
Ward 13, Size 58 Found 4 cycles
Ward 14, Size 115 Found 8 cycles
Ward 15, Size 96 Found 2 cycles
Ward 16, Size 74 Found 5 cycles
Ward 17, Size 91 Found 8 cycles
Ward 18, Size 83 Found 3 cycles
Ward 19, Size 99 Found 5 cycles
Ward 20, Size 127 Found 2 cycles
Ward 21, Size 111 Found 4 cycles
Ward 22, Size 99 Found 12 cycles
Ward 23, Size 109 Found 11 cycles
Ward 24, Size 59 Found 3 cycles
Ward 25, Size 96 Found 0 cycles


In [102]:
combined = list(itertools.chain.from_iterable(result_list))
result_df = pd.DataFrame.from_records(
    combined, columns = ['ward', 'cycles', 'scaled_cycles', 'removed_cycles']).groupby(
    'ward').sum().reset_index()
result_df = result_df.merge(pd.DataFrame.from_records(ward_n, columns = ['ward', 'N']))

In [103]:
result_df['cycle_availability'] = result_df['scaled_cycles']/(2 * result_df['N'] + 5)
result_df = result_df.merge(ward[['ward', 'area']])

In [104]:
result_df = result_df.merge(ward_int[['AREA_NAME', 'AREA_SHORT_CODE']].drop_duplicates(), left_on = ['ward'], 
                 right_on = ['AREA_SHORT_CODE']).rename(columns = {'AREA_NAME':'ward_name'}).drop(
    columns = ['AREA_SHORT_CODE'])
result_df['cycle_density'] = result_df['scaled_cycles']/result_df['area']
result_df.sort_values(by = 'cycle_availability', ascending = False)

,ward,cycles,scaled_cycles,removed_cycles,N,cycle_availability,area,ward_name,cycle_density
21,22,12,1.237624,0,99,0.006097,21.395977,Scarborough-Agincourt,0.057844
9,10,3,0.544554,0,50,0.005186,9.301089,Spadina-Fort York,0.058547
10,11,7,0.603960,0,73,0.004000,13.594704,University-Rosedale,0.044426
15,16,5,0.564356,0,74,0.003689,22.961198,Don Valley East,0.024579
12,13,4,0.396040,0,58,0.003273,5.861979,Toronto Centre,0.067561
16,17,8,0.594059,0,91,0.003177,24.416753,Don Valley North,0.024330
22,23,11,0.702970,0,109,0.003152,30.433949,Scarborough North,0.023098
23,24,3,0.356436,0,59,0.002898,26.114731,Scarborough-Guildwood,0.013649
3,4,8,0.514851,0,89,0.002813,15.324933,Parkdale-High Park,0.033596
0,1,14,0.811881,0,152,0.002627,48.352264,Etobicoke North,0.016791


In [105]:
result_df.sort_values(by = 'cycle_density', ascending = False)

,ward,cycles,scaled_cycles,removed_cycles,N,cycle_availability,area,ward_name,cycle_density
12,13,4,0.396040,0,58,0.003273,5.861979,Toronto Centre,0.067561
9,10,3,0.544554,0,50,0.005186,9.301089,Spadina-Fort York,0.058547
21,22,12,1.237624,0,99,0.006097,21.395977,Scarborough-Agincourt,0.057844
10,11,7,0.603960,0,73,0.004000,13.594704,University-Rosedale,0.044426
8,9,6,0.435644,0,87,0.002434,12.092764,Davenport,0.036025
3,4,8,0.514851,0,89,0.002813,15.324933,Parkdale-High Park,0.033596
4,5,10,0.752475,1,159,0.002330,24.973567,York South-Weston,0.030131
13,14,8,0.574257,0,115,0.002444,21.798056,Toronto-Danforth,0.026344
15,16,5,0.564356,0,74,0.003689,22.961198,Don Valley East,0.024579
16,17,8,0.594059,0,91,0.003177,24.416753,Don Valley North,0.024330


In [13]:
result_df.to_csv('cycles-' + period + '.csv', index = False)